# Livrable 1 : Classification Binaire
|Erwan Martin|Thibaut Liger-Hellard|Arnaud Maturel|Guillaume Le Cocguen|Victorien Goudeau|
|------------|---------------------|--------------|--------------------|-----------------|

**Contexte**:
L'objectif de ce livrable est classifier les photos des "non-photos" parmis les images du dataset.
La classification sera donc binaire.


# import des librairies

In [1]:
import os
from random import Random, random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import PIL
import random


In [3]:
print(torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NVIDIA GeForce RTX 2060


# Préparation des données
Afin de faire la classification binaire des images il va falloir préparer nos données en étiquettant nos images et en les mettant a la même échelle (scaling).

In [24]:
#data path definition
f = open('../data/path.txt', "r")
DATAPATH = f.read()
print(DATAPATH)

D:\CESI\A5\datascience\Projet\data


In [25]:
# dataset creation
labels = []
imgs = []

#creating labels table -----------------------------
for i in os.listdir(path=DATAPATH):
    if os.path.isdir(DATAPATH+"/"+i):
        labels.append(i)
#end creating labels table -----------------------------

#removing livrable2  from dataset------------------------
labels.remove("Dataset_L2")
labels.remove("Photo_2")
#end removing livrable2 from dataset------------------------


print(f"LABELS : {labels}")

#create BIG csv with image path + label
csv = open("../data/dataset_L1.csv", "w")
csv.write("pathname;label\n")
for label in labels:
    if label == "Photo":
        l = 1
    else:
        l=0
    for img in os.listdir(path=DATAPATH+"/"+label):
        csv.write(DATAPATH+"/"+label+"/"+img+";"+str(l)+"\n")
csv.close()

LABELS : ['Painting', 'Photo', 'Schematics', 'Sketch', 'Text']


maintenant, il faut le load dans un pandas, regardons ce que ca donne:

In [26]:
data = pd.read_csv("../data/dataset_L1.csv", sep=';')
data.head()

,pathname,label
0,D:\CESI\A5\datascience\Projet\data/Painting/pa...,0
1,D:\CESI\A5\datascience\Projet\data/Painting/pa...,0
2,D:\CESI\A5\datascience\Projet\data/Painting/pa...,0
3,D:\CESI\A5\datascience\Projet\data/Painting/pa...,0
4,D:\CESI\A5\datascience\Projet\data/Painting/pa...,0


In [31]:
img_size = 400

class ImagesDataset(Dataset):
    def __init__(self, path, file_path, labels, transform = None):
        super().__init__()
        self.df = pd.read_csv(file_path, sep=";", header=0)
        self.transform = transform
        self.class2index = {}
        for i in range(len(labels)):
            self.class2index.update({labels[i]:i})
            #{"label0":0 , "label1":1,...}
        

    def __getitem__(self, index):
        filename = self.df.loc[index]["pathname"]
        label = self.class2index[self.df.loc[index]["label"]]
        image = PIL.Image.open(path+filename)
        if self.transform is not None:
            image = self.transform(image)
        return image, label


    def __len__(self):
        return len(self.df)
    


#load into batched data
batch_size = 32

#transforms if we need
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])


full_dataset = ImagesDataset(path, "dataset.csv", labels, transform=transform)

#split
generator = torch.Generator().manual_seed(42)
train_set, test_set = random_split(full_dataset, [int(0.8*len(full_dataset)), int(0.2*len(full_dataset))], generator=generator)


train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)


print(f"TRAIN : {len((train_loader))}")
print(f"TEST : {len(test_loader)}")


D:\CESI\A5\datascience\Projet\data/Photo/photo_2921.jpg


AttributeError: in user code:

    File "C:\Users\erwan\AppData\Local\Temp\ipykernel_37564\176978609.py", line 9, in getImage  *
        img = PIL.Image.open(img_path)
    File "d:\anaconda3\envs\tensorflow\lib\site-packages\PIL\Image.py", line 3253, in open  **
        fp = io.BytesIO(fp.read())

    AttributeError: 'Tensor' object has no attribute 'read'


# CNN

In [28]:
class CNN(nn.Module):
    def __init__(self, img_witdh, num_labels):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)#3 color channels #6 outputs channels, 5 kernel size
        outputsize = self.get_outputSize(img_witdh, 5, 1 ,0)

        self.pool = nn.MaxPool2d(2,2) #kernel size 2, stride 2
        outputsize = self.get_outputSize(outputsize, 2, 2 ,0)

        self.conv2 = nn.Conv2d(6, 16, 5)#last output size from the convolution in input
        outputsize = self.get_outputSize(outputsize, 5, 1 ,0)
        
        self.pool = nn.MaxPool2d(2,2) #kernel size 2, stride 2
        outputsize = self.get_outputSize(outputsize, 2, 2 ,0)

        self.afterconv_size = 16*outputsize*outputsize

        self.fc1 = nn.Linear(self.afterconv_size, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_labels) 

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        #print("1 OK")
        x = self.pool(F.relu(self.conv2(x)))
        #print(x.shape)
        x = x.view(-1, self.afterconv_size) #flatten the tensor
        #print("3 OK")
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def get_outputSize(self, width_in, kernelSize, stride, padding):
        return int(((width_in - kernelSize + 2*padding  )/stride)+1)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 400, 400, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 400, 400, 15)      1140      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 200, 200, 15)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 200, 200, 30)      4080      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 100, 100, 30)     0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 300000)           

# Training

In [29]:
epochs=10
history =  model.fit(train_set, validation_data=test_set, epochs=epochs)

# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']

# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs_range = range(epochs)

# plt.figure(figsize=(16, 8))
# plt.subplot(1, 2, 1)
# plt.plot(epochs_range, acc, label='Training Accuracy')
# plt.plot(epochs_range, val_acc, label='Validation Accuracy')
# plt.legend(loc='lower right')
# plt.title('Training and Validation Accuracy')

# plt.subplot(1, 2, 2)
# plt.plot(epochs_range, loss, label='Training Loss')
# plt.plot(epochs_range, val_loss, label='Validation Loss')
# plt.legend(loc='upper right')
# plt.title('Training and Validation Loss')
# plt.show()

Epoch 1/10


d:\anaconda3\envs\tensorflow\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


 70/906 [=>............................] - ETA: 1:02 - loss: 1.0299 - accuracy: 0.7281

InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]
	 [[IteratorGetNext/_2]]
  (1) INVALID_ARGUMENT:  Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_5133]